In [180]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup


## Pulling all SEC 13 F links

In [181]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Constants
CIK = "1536411"
SEC_API_URL = f"https://data.sec.gov/submissions/CIK000{CIK}.json"

# Headers for the SEC API request
headers = {'User-Agent': "bfassnacht17@gmail.com"}

base_url = "https://sec.gov"

# Function to get 13F filings
def get_13f_filings(cik):
    url = f"https://data.sec.gov/submissions/CIK000{cik}.json"
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        
        # Extract recent filings data
        forms = data['filings']['recent']['form']
        accession_numbers = data['filings']['recent']['accessionNumber']
        filing_dates = data['filings']['recent']['filingDate']
        
        # Create a list to store filing data
        filings_data = []
        
        # Loop through filings and filter for 13F-HR forms
        for i, form in enumerate(forms):
            if form == "13F-HR":
                accession_number = accession_numbers[i]
                filing_date = filing_dates[i]
                
                # Construct the link to the filing's index page
                filing_url = f"https://www.sec.gov/Archives/edgar/data/{int(cik)}/{accession_number}/{accession_number}-index.htm"
                
                # Parse the filing index page to find the 13F XML file link
                form_13f_url = get_form_13f_url(filing_url)
                
                # Append the data to the list
                filings_data.append({
                    'Form': form,
                    'Filing Date': filing_date,
                    'Filing Index URL': filing_url,
                    'Form 13F URL': form_13f_url
                })
        
        # Convert the list to a DataFrame
        df_filings = pd.DataFrame(filings_data)
        return df_filings
    else:
        print("Failed to retrieve data.")
        return None

# Function to extract the Form 13F XML file URL from the index page
def get_form_13f_url(index_url):
    response = requests.get(index_url, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Look for the link to the Form 13F XML file
        links = soup.find_all('a')
        
        for link in links:
            href = link.get('href')
            if href and ('slform13f' in href.lower() and href.endswith('.xml') and 'primary_doc' not in href.lower()):
                # Construct the full URL
               
                full_url = f"{base_url}/{href}"
                return full_url
    
    return None

# Get 13F filings
df_13f = get_13f_filings(CIK)

if df_13f is not None:
    df_13f


In [182]:
df_13f_filtered = df_13f[df_13f["Form 13F URL"].notna()]

## Pulling Holdings into data frame

In [183]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# URL of the XML file
url = "https://sec.gov/Archives/edgar/data/1536411/000153641124000007/xslForm13F_X02/form13f_20240630.xml"

dfs = []

for index, row in df_13f_filtered.iterrows():

    url = row["Form 13F URL"]
    filing_date = row["Filing Date"]
    
    # Fetching the XML content from the URL
    response = requests.get(url, headers=headers)
    xml_content = response.content

    # Create a BeautifulSoup object
    soup = BeautifulSoup(xml_content, 'html.parser')

    # Find the table element containing the data
    table = soup.find('table', summary="Form 13F-NT Header Information")

    # Extract the column headers
    header_row = table.find('tr')
    header_cells = header_row.find_all('td')
    column_headers = [cell.text.strip() for cell in header_cells]

    # Extract the data rows
    data_rows = table.find_all('tr')[1:]  # Skip the header row
    data = []
    for row in data_rows:
        cells = row.find_all('td')
        row_data = [cell.text.strip() for cell in cells]
        data.append(row_data)   

    # Create a pandas DataFrame
    df = pd.DataFrame(data[2:], columns=data[1])

    # Handle non-breaking space characters (if present)
    df = df.replace('\xa0', '', regex=True)

    dfs.append([filing_date, df])



In [186]:
dfs

[['2024-08-14',
                 NAME OF ISSUER TITLE OF CLASS      CUSIP FIGI  \
  0                   Adobe Inc            COM  00724F101        
  1                    Aes Corp            COM  00130H105        
  2                   Apple Inc            COM  037833100        
  3                 Arcellx Inc   COMMON STOCK  03940C100        
  4         Arista Networks Inc            COM  040413106        
  ..                        ...            ...        ...  ...   
  59                Vistra Corp            COM  92840M102        
  60                     Wabtec            COM  929740108        
  61               Woodward Inc            COM  980745103        
  62       Ypf Sociedad Anonima  SPON ADR CL D  984245100        
  63  Zoominfo Technologies Inc   COMMON STOCK  98980F104        
  
     (to the nearest dollar)    PRN AMT PRN CALL DISCRETION MANAGER       SOLE  \
  0                   20,380     36,685  SH            DFND             36,685   
  1                    5,

In [185]:
df_13f_filtered

,Form,Filing Date,Filing Index URL,Form 13F URL
0,13F-HR,2024-08-14,https://www.sec.gov/Archives/edgar/data/153641...,https://sec.gov//Archives/edgar/data/1536411/0...
1,13F-HR,2024-05-15,https://www.sec.gov/Archives/edgar/data/153641...,https://sec.gov//Archives/edgar/data/1536411/0...
2,13F-HR,2024-02-14,https://www.sec.gov/Archives/edgar/data/153641...,https://sec.gov//Archives/edgar/data/1536411/0...
3,13F-HR,2023-11-14,https://www.sec.gov/Archives/edgar/data/153641...,https://sec.gov//Archives/edgar/data/1536411/0...
4,13F-HR,2023-08-14,https://www.sec.gov/Archives/edgar/data/153641...,https://sec.gov//Archives/edgar/data/1536411/0...
5,13F-HR,2023-05-15,https://www.sec.gov/Archives/edgar/data/153641...,https://sec.gov//Archives/edgar/data/1536411/0...
6,13F-HR,2023-02-14,https://www.sec.gov/Archives/edgar/data/153641...,https://sec.gov//Archives/edgar/data/1536411/0...
7,13F-HR,2022-11-14,https://www.sec.gov/Archives/edgar/data/153641...,https://sec.gov//Archives/edgar/data/1536411/0...
8,13F-HR,2022-08-15,https://www.sec.gov/Archives/edgar/data/153641...,https://sec.gov//Archives/edgar/data/1536411/0...
9,13F-HR,2022-05-16,https://www.sec.gov/Archives/edgar/data/153641...,https://sec.gov//Archives/edgar/data/1536411/0...
